In [ ]:
import json
import difflib

from matplotlib import pyplot as plt
import matplotlib.cm as cm
import matplotlib
import matplotlib.colors as mcolors
from tqdm import tqdm

from network import *
from analyze_graph import *
import smopy
import pandas as pd


In [ ]:
network = Network()
network.set_stops("network.db")
stops_data = pd.read_csv("stops.csv", index_col="DHID")

In [ ]:
kassel = find_closest_station_id_by_name("Kassel Holländischer Platz", stops_data)
marburg = find_closest_station_id_by_name("Marburg Hauptbahnhof", stops_data)
giessen = find_closest_station_id_by_name("Gießen Bahnhof", stops_data)
darmstadt = find_closest_station_id_by_name("Darmstadt Schloss", stops_data)

In [ ]:
stops_data.loc[darmstadt]

In [ ]:
start_station = darmstadt
time_limit = 30
start_time = time_to_minutes("09:00")
reachable = network.get_reachable_stations_in_time(start_station, start_time, time_limit)

In [ ]:
min_lat = 180
min_long = 180
max_lat = -180
max_long = -180

coordinates = []
for station in reachable:
    lat = stops_data.loc[station]["Latitude"]
    lon = stops_data.loc[station]["Longitude"]
    coordinates.append((lat, lon))

    min_lat = min(min_lat, lat)
    min_long = min(min_long, lon)
    max_lat = max(max_lat, lat)
    max_long = max(max_long, lon)

# area to plot
map_box = (min_lat, min_long, max_lat, max_long)
print(map_box)
map = smopy.Map(map_box)

# figsize is used for resolution
ax = map.show_mpl(figsize=(12, 12))

# TODO annotation is not working correctly
ax.annotate(
    "Reachable from %s (%s) until %s (%i stops)" % (
        stops_data.loc[start_station, "Name"],
        minutes_to_time(start_time),
        minutes_to_time(start_time + time_limit),
        len(reachable)),
    xy=(0.5, 1.02),  # Position relative to axes (centered above the map)
    xycoords="axes fraction",
    fontsize=24,
    ha="center",  # Center horizontally
    va="bottom",  # Position below the top edge
)

for lat, lon in coordinates:
    x, y = map.to_pixels(lat, lon)
    ax.plot(x, y, 'or', ms=10, mew=2)

In [ ]:
map_box = (51.23520, 9.39316, 51.38409, 9.61776) # area around Kassel
map_box = (50.756976, 8.703221, 50.848513, 8.829656) # area around Marburg
map_box = (50.522775, 8.601746, 50.617075, 8.7374) # area around Gießen
map_box = (49.8388, 8.560719, 49.931479, 8.750582)  # area around Darmstadt
map = smopy.Map(map_box)
# select stops in that area
in_area = stops_data[(stops_data["Latitude"] > map_box[0]) &
                     (stops_data["Latitude"] < map_box[2]) &
                     (stops_data["Longitude"] > map_box[1]) &
                     (stops_data["Longitude"] < map_box[3])].copy()

In [ ]:
end_of_day = 3 * 60 + 50  #03:59


# a bus leaving at 0:30 is part of the old daythat is your ticket from the previous day is still valid

def get_early(stop_id):
    earliest_depature = 23 * 60 + 59  # 23:59
    for stop_id, timetable in network.stops[stop_id].items():
        timetable.sort()
        idx = 0
        while idx < len(timetable) and timetable[idx][0] < end_of_day:
            idx += 1
        # found the next departure
        if idx < len(timetable):
            earliest_depature = min(timetable[idx][0], earliest_depature)
    return earliest_depature


# to_series as, as the stop_id is the index
in_area["first_departure"] = in_area.index.to_series().apply(get_early)

In [ ]:
from matplotlib.lines import Line2D

in_area["first_departure_hour"] = in_area["first_departure"] // 60  # only the hour

unique_hours = sorted(in_area["first_departure_hour"].unique())
n_colors = len(unique_hours)
colors = cm.get_cmap("tab10", n_colors)  # Or "tab20", "Set3", etc.
hour_to_color = {hour: colors(i) for i, hour in enumerate(unique_hours)}

#map = smopy.Map(map_box)

# figsize is used for resolution
ax = map.show_mpl(figsize=(12, 12))

# TODO annotation is not working correctly
ax.annotate(
    "Earliest Depature (after 03:50)",
    xy=(0.5, 1.02),  # Position relative to axes (centered above the map)
    xycoords="axes fraction",
    fontsize=24,
    ha="center",  # Center horizontally
    va="bottom",  # Position below the top edge
)

for _, row in in_area.iterrows():
    x, y = map.to_pixels(row["Latitude"], row["Longitude"])
    ax.plot(x, y, 'or', color=hour_to_color[row["first_departure_hour"]], ms=10, mew=2)

legend_elements = [
    Line2D([0], [0], marker='o', color='w', label=f"{hour}:00",
           markerfacecolor=color, markersize=10)
    for hour, color in hour_to_color.items() if hour != 23
]
ax.legend(handles=legend_elements, title="First Departure Hour")

In [ ]:
def get_depatures(stop_id):
    result = []
    for stop_id, timetable in network.stops[stop_id].items():
        timetable.sort()
        for depature, _, _, _ in timetable:
            result.append(depature)
    return result


departures = get_depatures(darmstadt)  # list of minutes since midnight
# Convert to hours
departure_hours = [t // 60 for t in departures if t < 1440]

plt.hist(departure_hours, bins=range(25), edgecolor='black', color='skyblue', align='left')
plt.xticks(range(24))
plt.xlabel("Hour of Day")
plt.ylabel("Number of Departures")
plt.title(f"Departures per Hour at {stops_data.loc[darmstadt]['Name']}")
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import inset_axes


def count_daytime_departures(stop_id):
    departures = get_depatures(stop_id)
    return sum(9 <= (t // 60) < 18 for t in departures)  # counts hours 9–17


in_area["daytime_departures"] = in_area.index.map(count_daytime_departures)
#in_area["daytime_departures"] =in_area["daytime_departures"] / 8 # per hour
in_area_with_depatures = in_area[in_area["daytime_departures"] > 0]  # dont picture stations only used at night (or not at all)

norm = mcolors.LogNorm(
    vmin=in_area_with_depatures["daytime_departures"].min(),
    vmax=in_area_with_depatures["daytime_departures"].max()
)
cmap = matplotlib.colormaps.get_cmap("plasma")  # or "plasma", "virdis", etc.

#map = smopy.Map(map_box)

# figsize is used for resolution
ax = map.show_mpl(figsize=(12, 12))

ax.annotate(
    "Number of Departures from 9 to 17:00",
    xy=(0.5, 1.02),  # Position relative to axes (centered above the map)
    xycoords="axes fraction",
    fontsize=24,
    ha="center",  # Center horizontally
    va="bottom",  # Position below the top edge
)

for _, row in in_area_with_depatures.iterrows():
    lat, lon = row["Latitude"], row["Longitude"]
    x, y = map.to_pixels(lat, lon)

    count = row["daytime_departures"]
    color = cmap(norm(count))

    ax.plot(x, y, 'o', color=color, ms=10, mew=0.5)

cax = inset_axes(ax,
                 width="5%",  # width of colorbar relative to parent axes
                 height="50%",  # height of colorbar relative to parent axes
                 loc='upper right',
                 borderpad=2)

sm = cm.ScalarMappable(norm=norm, cmap=cmap)
plt.colorbar(sm, cax=cax, label="Number of Denatures (9–17h)")

In [ ]:
def get_num_connected_nodes(stop_id):
    #for iid , _ in network.stops[stop_id].items():
    #    print(stops_data.loc[iid]["Name"])
    return len(network.stops[stop_id].items())

in_area["num_connections"] = in_area.index.to_series().apply(get_num_connected_nodes)
in_area =in_area[in_area["num_connections"]>0]

In [ ]:
norm = mcolors.LogNorm(
    vmin=in_area["num_connections"].min(),
    vmax=in_area["num_connections"].max()
)
cmap = matplotlib.colormaps.get_cmap("plasma")  # or "plasma", "virdis", etc.

#map = smopy.Map(map_box)

# figsize is used for resolution
ax = map.show_mpl(figsize=(12, 12))

ax.annotate(
    "Number of connected nodes (also counting stations outside of shown area)",
    xy=(0.5, 1.02),  # Position relative to axes (centered above the map)
    xycoords="axes fraction",
    fontsize=24,
    ha="center",  # Center horizontally
    va="bottom",  # Position below the top edge
)

for _, row in in_area.iterrows():
    lat, lon = row["Latitude"], row["Longitude"]
    x, y = map.to_pixels(lat, lon)

    count = row["num_connections"]
    color = cmap(norm(count))

    ax.plot(x, y, 'o', color=color, ms=10, mew=0.5)

cax = inset_axes(ax,
                 width="5%",  # width of colorbar relative to parent axes
                 height="50%",  # height of colorbar relative to parent axes
                 loc='upper right',
                 borderpad=2)

sm = cm.ScalarMappable(norm=norm, cmap=cmap)
plt.colorbar(sm, cax=cax, label="Number of connected nodes")

In [ ]:
# number of vehicles on this network section per day
network_frequencies = {}
for stop_id, row in in_area.iterrows():
    for connected_stop, timetable in network.stops[stop_id].items():
        if (stop_id, connected_stop) in network_frequencies:
            network_frequencies[(stop_id, connected_stop)] += len(timetable)
        elif (connected_stop, stop_id) in network_frequencies:
            network_frequencies[(connected_stop, stop_id)] += len(timetable)
        else:
            # add new entry
            network_frequencies[(connected_stop, stop_id)] = len(timetable)

network_frequencies


In [ ]:
freq_values = [v for (begin, end), v in network_frequencies.items() if
               v > 0 and begin in in_area.index and end in in_area.index]  # filter out 0 aka no connection - should not happen anyway, and only show the stations in selected area
norm = mcolors.LogNorm(
    vmin=min(freq_values),
    vmax=max(freq_values)
)
cmap = matplotlib.colormaps.get_cmap("plasma")  # or "plasma", "virdis", etc.

#map = smopy.Map(map_box)

# figsize is used for resolution
ax = map.show_mpl(figsize=(12, 12))

ax.annotate(
    "Number of Vehicles on Route per Day",
    xy=(0.5, 1.02),  # Position relative to axes (centered above the map)
    xycoords="axes fraction",
    fontsize=24,
    ha="center",  # Center horizontally
    va="bottom",  # Position below the top edge
)

for (start, stop), freq in network_frequencies.items():
    # cleaner visualization as some single or twice a day trips look wired on the map (this happens mostly in the night)
    #if start in in_area.index and stop in in_area.index:
    if freq > 2  and start in in_area.index and stop in in_area.index:
        lat1, lon1 = in_area.loc[start,"Latitude"], in_area.loc[start,"Longitude"]
        x1, y1 = map.to_pixels(lat1, lon1)
        lat2, lon2 = in_area.loc[stop,"Latitude"], in_area.loc[stop,"Longitude"]
        x2, y2 = map.to_pixels(lat2, lon2)
        color = cmap(norm(freq))
        ax.plot([x1, x2], [y1, y2], '-', color=color, linewidth=2)

cax = inset_axes(ax,
                 width="5%",  # width of colorbar relative to parent axes
                 height="50%",  # height of colorbar relative to parent axes
                 loc='upper right',
                 borderpad=2)

sm = cm.ScalarMappable(norm=norm, cmap=cmap)
plt.colorbar(sm, cax=cax, label="Number of Vehicles per Day")


In [ ]:
from matplotlib.cm import ScalarMappable

reachable = dict()
time_limits = [0,15,30,45,60,75,90]

for time in time_limits:
    reachable[time] = network.get_reachable_stations_in_time(start_station, start_time, time)

# Define color map
cmap = cm.get_cmap("viridis", len(time_limits)) # discrete colors
norm = mcolors.BoundaryNorm(time_limits, cmap.N)

#map = smopy.Map(map_box)

# figsize is used for resolution
ax = map.show_mpl(figsize=(12, 12))

ax.annotate(
   "Reachable from %s (%s)" % (
        stops_data.loc[start_station, "Name"],
        minutes_to_time(start_time)),
    xy=(0.5, 1.02),  # Position relative to axes (centered above the map)
    xycoords="axes fraction",
    fontsize=24,
    ha="center",  # Center horizontally
    va="bottom",  # Position below the top edge
)

for station_id, row in in_area.iterrows():
    lat, lon = row["Latitude"], row["Longitude"]
    x, y = map.to_pixels(lat, lon)

    # get time
    reached_by = None
    for time in time_limits:
        if station_id in reachable[time]:
            reached_by = time
            break
    if reached_by is not None:
        color = cmap(norm(reached_by))
        ax.plot(x, y, 'o', color=color, ms=10, mew=0.5)

cax = inset_axes(ax,
                 width="5%",  # width of colorbar relative to parent axes
                 height="50%",  # height of colorbar relative to parent axes
                 loc='upper right',
                 borderpad=2)

sm = ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])  # required for colorbar


# Midpoints between time limits for tick positions
tick_positions = [(time_limits[i] + time_limits[i+1]) / 2 for i in range(len(time_limits) - 1)]
tick_labels = [f"{int(t)}" for t in time_limits[1:]]

cbar = plt.colorbar(sm, cax=cax, boundaries=time_limits, ticks=tick_positions)
cbar.ax.set_yticklabels(tick_labels)
cbar.set_label('Minutes to Reach', fontsize=14)

In [ ]:
import shapely
from matplotlib.cm import ScalarMappable
import alphashape
from descartes import PolygonPatch


# Define color map
cmap = cm.get_cmap("viridis", len(time_limits)) # discrete colors
norm = mcolors.BoundaryNorm(time_limits, cmap.N)

#map = smopy.Map(map_box)

# figsize is used for resolution
ax = map.show_mpl(figsize=(12, 12))

ax.annotate(
   "Reachable from %s (%s)" % (
        stops_data.loc[start_station, "Name"],
        minutes_to_time(start_time)),
    xy=(0.5, 1.02),  # Position relative to axes (centered above the map)
    xycoords="axes fraction",
    fontsize=24,
    ha="center",  # Center horizontally
    va="bottom",  # Position below the top edge
)


time_limits = [0,15,30,45,60,75,90]


for time in reversed(time_limits):
    reachable = network.get_reachable_stations_in_time(start_station, start_time, time)
    points = []
    color = cmap(norm(time))
    for station_id, row in in_area.iterrows():
        if station_id in reachable:
            lat, lon = row["Latitude"], row["Longitude"]
            x, y = map.to_pixels(lat, lon)
            points.append((x, y))
    if len(points) >= 3:  # Need at least 3 for a polygon
        # slow calculation, uses suitable alpha parameter, for nicer looking plot
        alpha_shape = alphashape.alphashape(points)
        # fast calculation (simple convex hull)
        #alpha_shape = alphashape.alphashape(points, alpha=0)
        if(isinstance(alpha_shape, (shapely.geometry.Polygon,shapely.geometry.MultiPolygon))):
            ax.add_patch(PolygonPatch(alpha_shape,fc=color, ec=color, alpha=0.3))

cax = inset_axes(ax,
                 width="5%",  # width of colorbar relative to parent axes
                 height="50%",  # height of colorbar relative to parent axes
                 loc='upper right',
                 borderpad=2)

sm = ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])  # required for colorbar


# Midpoints between time limits for tick positions
tick_positions = [(time_limits[i] + time_limits[i+1]) / 2 for i in range(len(time_limits) - 1)]
tick_labels = [f"{int(t)}" for t in time_limits[1:]]

cbar = plt.colorbar(sm, cax=cax, boundaries=time_limits, ticks=tick_positions)
cbar.ax.set_yticklabels(tick_labels)
cbar.set_label('Minutes to Reach', fontsize=14)